<a href="https://colab.research.google.com/github/GrizzlyToast/ML_Practise/blob/main/BERT_tokenizer_TwitterSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up Environemnt

In [1]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [2]:
! kaggle datasets download -d kazanova/sentiment140
! unzip -qq sentiment140.zip
! mv training.1600000.processed.noemoticon.csv train.csv

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.08GB/s]


# Installing Dependencies

In [3]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

from transformers import BertTokenizerFast, BertModel, DataCollatorWithPadding
from datasets import Dataset

# Data Preprocessing

In [5]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv("train.csv",
                   header=None,
                   names=cols,
                   engine="python",
                   encoding="latin1")

In [6]:
data.drop(["id", "date", "query", "user"], axis=1, inplace=True)

In [7]:
data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
def clean_tweet(tweet):
    """ Cleans text from tweets by removing tags, urls and special characters """
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^A-za-z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [9]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [10]:
data['text'] = data_clean
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1
data.head(5)

,sentiment,text
0,0,Awww that's a bummer. You shoulda got David C...
1,0,is upset that he can't update his Facebook by ...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,no it's not behaving at all. i'm mad. why am ...


In [11]:
data = data.rename(columns={'sentiment': 'labels'})
dataset = Dataset.from_pandas(data)
print(dataset)

Dataset({
    features: ['labels', 'text'],
    num_rows: 1600000
})


In [14]:
sentence = dataset[0]['text']
print(sentence)

 Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D


# Tokenization and Dataset Creation

In [52]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

## Tokenization

In [15]:
# Tokenizer Example
print(sentence)
print(tokenizer(sentence))
print(tokenizer.convert_ids_to_tokens(tokenizer(sentence)['input_ids']))

 Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D
{'input_ids': [101, 22091, 2860, 2860, 2008, 1005, 1055, 1037, 26352, 5017, 1012, 2017, 2323, 2050, 2288, 2585, 12385, 1997, 2353, 2154, 2000, 2079, 2009, 1012, 1040, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]']


In [16]:
dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True), batched=True)

Map:   0%|          | 0/1600000 [00:00<?, ? examples/s]

## Batch Creation

In [17]:
batched_dataset = dataset.to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)

In [46]:
next(iter(batched_dataset))

{'labels': <tf.Tensor: shape=(32,), dtype=int64, numpy=
 array([1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1])>,
 'input_ids': <tf.Tensor: shape=(32, 40), dtype=int64, numpy=
 array([[ 101, 2079, 2009, ...,    0,    0,    0],
        [ 101, 2179, 2009, ...,    0,    0,    0],
        [ 101, 2129, 2064, ...,    0,    0,    0],
        ...,
        [ 101, 2234, 2046, ...,    0,    0,    0],
        [ 101, 7143, 2160, ...,    0,    0,    0],
        [ 101,  999, 2054, ...,    0,    0,    0]])>,
 'token_type_ids': <tf.Tensor: shape=(32, 40), dtype=int64, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>,
 'attention_mask': <tf.Tensor: shape=(32, 40), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]

In [ ]:
NB_BATCHES = len(batched_dataset)
NB_BATCHES_TEST = NB_BATCHES // 10
test_dataset = batched_dataset.take(NB_BATCHES_TEST)
train_dataset = batched_dataset.skip(NB_BATCHES_TEST)

# Model

In [52]:
class DCNN(tf.keras.Model):
    def ___init__(self,
                  vocab_size,
                  emb_dim=128,
                  nb_filters=50,
                  FFN_units=512,
                  nb_classes=2,
                  dropout_rate=0.1,
                  training=False,
                  name="dcnn"):
        super(DCNN, self).__init__(name=name)
        self.embedding = layers.Embedding(vocab_size, emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=3,
                                    padding="valid",
                                    activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=4,
                                    padding="valid",
                                    activation="relu")
        self.pool = layers.GlobalMaxPool1D()